In [ ]:
print("AI SEARCH")

AI SEARCH


In [ ]:
# visualization libraries
!pip install seaborn altair bokeh plotly hvplot folium geopandas ee shap

# NLP libraries

!pip install torch torchvision torchtext torchnlp skorch captum spacy huggingface transformers sentence-transformers allennlp-all textblob nltk gensim polyglot pynlpl

In [3]:
text = [
    "Artificial Intelligence (AI) is transforming industries by automating processes, enhancing decision-making, and enabling innovative solutions.",
    "The sun rises in the east and sets in the west, a fundamental concept in astronomy.",
    "In 1956, John McCarthy coined the term 'Artificial Intelligence' at the Dartmouth Conference, marking the beginning of AI as a field of study.",
    "Deep learning models, particularly transformers, have revolutionized natural language processing tasks such as translation, text generation, and sentiment analysis.",
    "Once upon a time in a small village, there lived a young boy who dreamed of exploring the world. He spent his days reading about distant lands and imagining adventures beyond the mountains.",
    "Data privacy and security are critical concerns in the digital age, where companies collect vast amounts of user information.",
    "A cat sat on the windowsill, staring at the birds outside with intense curiosity.",
    "Quantum computing promises to revolutionize problem-solving by leveraging quantum states, offering potential breakthroughs in cryptography and optimization.",
    "Climate change is a pressing global issue that requires immediate action. Rising temperatures, melting glaciers, and extreme weather events highlight the urgency of adopting sustainable practices and reducing carbon emissions.",
    "The quick brown fox jumps over the lazy dog.",
    "In a groundbreaking discovery, scientists have found evidence of water on Mars, opening new possibilities for future space exploration and the search for extraterrestrial life.",
    "Startups often face challenges such as securing funding, finding product-market fit, and scaling operations efficiently.",
    "The mysterious island was shrouded in mist, hiding secrets that had been forgotten for centuries.",
    "Machine learning is a subset of AI that allows computers to learn patterns from data without being explicitly programmed.",
    "A sudden storm erupted, drenching the streets in heavy rain and causing people to run for shelter.",

    # Long Paragraphs
    "The concept of consciousness has intrigued philosophers, scientists, and thinkers for centuries. Despite advancements in neuroscience and psychology, the true nature of human awareness remains a mystery. Some theories suggest that consciousness emerges from complex neural interactions, while others argue that it is a fundamental aspect of reality. Theories like panpsychism propose that even the smallest particles of matter might possess some form of awareness, challenging the traditional materialistic view of the universe. As artificial intelligence and cognitive sciences advance, researchers continue to explore whether machines can ever achieve true consciousness, or if it remains an inherently biological phenomenon.",

    "The history of space exploration is a testament to human ingenuity and curiosity. From the early days of rocketry in the mid-20th century to the present era of Mars rovers and deep-space telescopes, humanity has made significant strides in understanding the cosmos. The Apollo 11 mission in 1969 marked a historic moment as humans set foot on the Moon, proving that space travel was not just a dream. In the decades that followed, numerous robotic missions have explored distant planets, sending back invaluable data. With advancements in space technology, agencies like NASA, ESA, and private companies such as SpaceX are now working towards manned missions to Mars, pushing the boundaries of exploration further than ever before.",

    "The Renaissance was a period of profound cultural, artistic, and intellectual growth that spanned roughly from the 14th to the 17th century. Originating in Italy, this era witnessed a revival of classical knowledge, a flourishing of the arts, and a shift towards humanist philosophy. Great minds such as Leonardo da Vinci and Michelangelo revolutionized painting and sculpture, while figures like Galileo and Copernicus laid the foundation for modern science. The printing press, invented by Johannes Gutenberg, played a crucial role in spreading knowledge, enabling literacy to flourish. The Renaissance not only transformed Europe but also set the stage for future advancements in science, politics, and the arts, shaping the world we know today."
]


In [5]:
# from huggingface_hub import login
# from google.colab import userdata
# HF_Token = userdata.get('HF_Token')
# login(token=HF_Token)

In [ ]:
from sentence_transformers import SentenceTransformer

MODEL = "msmarco-distilbert-base-v4"
embedder = SentenceTransformer(MODEL)

In [9]:
embeddings = embedder.encode(text,convert_to_tensor=True)
embeddings.shape

torch.Size([18, 768])

In [14]:
query_text = "making computer think like human"
query = embedder.encode(query_text, convert_to_tensor=True)

In [15]:
from sentence_transformers import util
search_results = util.semantic_search(query,embeddings,top_k=3)
search_results

[[{'corpus_id': 0, 'score': 0.36625856161117554},
  {'corpus_id': 13, 'score': 0.3088148832321167},
  {'corpus_id': 2, 'score': 0.2613815665245056}]]

In [16]:
for index, result in enumerate(search_results[0]):
  print("_"*90)
  print(f"Search Rank: {index}, Relevance Score: {result['score']}")
  print(text[result["corpus_id"]])

__________________________________________________________________________________________
Search Rank: 0, Relevance Score: 0.36625856161117554
Artificial Intelligence (AI) is transforming industries by automating processes, enhancing decision-making, and enabling innovative solutions.
__________________________________________________________________________________________
Search Rank: 1, Relevance Score: 0.3088148832321167
Machine learning is a subset of AI that allows computers to learn patterns from data without being explicitly programmed.
__________________________________________________________________________________________
Search Rank: 2, Relevance Score: 0.2613815665245056
In 1956, John McCarthy coined the term 'Artificial Intelligence' at the Dartmouth Conference, marking the beginning of AI as a field of study.


Visual Search

In [20]:
from PIL import Image

CLIP_MODEL = "clip-ViT-B-32"
embedder = SentenceTransformer(CLIP_MODEL)

short_sentences = [
    "A similing dog","Car on highway","Elephant in the field","Attention is all you need","world cup winner","Golden retrievers are little children"
]

embeddings = embedder.encode(short_sentences,convert_to_tensor=True)
embeddings.shape

torch.Size([6, 512])

In [18]:
query = embedder.encode(Image.open("/content/dog.jpg"))

In [21]:
from sentence_transformers import util

search_results = util.semantic_search(query,embeddings,top_k=3)
search_results

[[{'corpus_id': 5, 'score': 0.2793245315551758},
  {'corpus_id': 0, 'score': 0.25686395168304443},
  {'corpus_id': 3, 'score': 0.2093714475631714}]]

In [22]:
for index, result in enumerate(search_results[0]):
  print("_"*90)
  print(f"Search Rank: {index}, Relevance Score: {result['score']}")
  print(short_sentences[result["corpus_id"]])

__________________________________________________________________________________________
Search Rank: 0, Relevance Score: 0.2793245315551758
Golden retrievers are little children
__________________________________________________________________________________________
Search Rank: 1, Relevance Score: 0.25686395168304443
A similing dog
__________________________________________________________________________________________
Search Rank: 2, Relevance Score: 0.2093714475631714
Attention is all you need


Text as query and search images

In [23]:
from PIL import Image
import glob
import torch
import pickle
import zipfile
from IPython.display import display
from IPython.display import Image as IPImage
import os
from tqdm.autonotebook import tqdm


In [27]:
img_folder = "/content/photos/"

if not os.path.exists(img_folder) or len(os.listdir(img_folder)) == 0 :
  os.makedirs(img_folder,exist_ok=True)

  photo_filename = "unsplash-25k-photos.zip"

  if not os.path.exists(photo_filename):
    util.http_get("http://sbert.net/datasets/"+ photo_filename,photo_filename)

  with zipfile.ZipFile(photo_filename,"r") as zf:
    for member in tqdm(zf.infolist(),desc="Extracting"):
      zf.extract(member,img_folder)

Extracting:   0%|          | 0/24996 [00:00<?, ?it/s]

In [ ]:
model = SentenceTransformer("clip-ViT-B-32")

img_names = list(glob.glob("/content/photos/*.jpg"))
print("Images: ",len(img_names))

img_emb = model.encode([
    Image.open(filepath) for filepath in img_names
],
   batch_size=128,convert_to_tensor=True,show_progess_bar=True )


Images:  24996


In [ ]:
def search(query,k=3):
  query_emb = model.encode([query],convert_to_tensor=True,show_progess_bar=False)
  hits = util.semantic_search(query_emb,img_emb,top_k=k)[0]

  print("Query: ")
  display(query)
  for hit in hits:
    print(img_names[hit["corpus_id"]])
    display(IPImage(os.path.join(img_folder,img_names[hit["corpus_id"]]),width=200))



In [ ]:
search("love a dog ")